In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *
from Recommenders.BaseRecommender import BaseRecommender
import scipy.sparse as sps

c:\Users\ES3\.conda\envs\RecSysFramework\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [3]:
# 0.08254950935282762
slimel_study = optuna.load_study(
    study_name="SLIMElastic",
    storage=get_database_url(),
)

# 0.07018143152516958
slimbpr_study = optuna.load_study(
    study_name="SLIMBPR",
    storage=get_database_url(),
)

# 0.08248515814658722
p3alpha_study = optuna.load_study(
    study_name="P3Alpha",
    storage=get_database_url(),
)

# 0.082429130748692
easer_study = optuna.load_study(
    study_name="Easer",
    storage=get_database_url(),
)

# 0.06470798788105654
userknncf_study = optuna.load_study(
    study_name="UserKNNCF",
    storage=get_database_url(),
)

# 0.07684279728158312
itemknncf_study = optuna.load_study(
    study_name="ItemKNNCF",
    storage=get_database_url(),
)

# 0.08511222934213017
p3beta_study = optuna.load_study(
    study_name="P3Beta",
    storage=get_database_url(),
)

In [4]:
recommenders = [
    {
        "recommender": ItemKNNCFRecommender,
        "params": itemknncf_study.best_params,
    },
    {
        "recommender": UserKNNCFRecommender,
        "params": userknncf_study.best_params,
    },
    {
        "recommender": RP3betaRecommender,
        "params": p3beta_study.best_params,
    },
    {
        "recommender": EASE_R_Recommender,
        "params": easer_study.best_params,
    },
    {
        "recommender": P3alphaRecommender,
        "params": p3alpha_study.best_params,
    },
    {
        "recommender": SLIM_BPR_Cython,
        "params": slimbpr_study.best_params,
    },
    {
        "recommender": MultiThreadSLIM_SLIMElasticNetRecommender,
        "params": slimel_study.best_params,
    },
]

In [5]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, data_train, recommenders):
        super(ScoresHybridRecommender, self).__init__(data_train)

        self.data_train = sps.csr_matrix(data_train)
        self.recommenders = recommenders
        
    def prefit(self):
        for rec in self.recommenders:
            rec["recommender"] = rec["recommender"](self.data_train)
            rec["recommender"].fit(**rec["params"])
        
        
    def fit(self, weights):
        self.weights = weights      


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights = []
        for rec in self.recommenders:
            item_weights.append(rec["recommender"]._compute_item_score(user_id_array, items_to_compute))

        item_weights = sum([a*b for a,b in zip(item_weights, self.weights)])
        return item_weights

In [6]:
scores_hybrid_study = optuna.create_study(
    study_name="ScoresHybrid2",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-10 21:31:57,732] Using an existing study with name 'ScoresHybrid2' instead of creating a new one.


In [7]:
scores_hybrid_recommender = ScoresHybridRecommender(data_train, recommenders)
scores_hybrid_recommender.prefit()

def objective(trial):
    weights = []
    for i in range(len(recommenders)):
        weights.append(trial.suggest_uniform("weight_{}".format(i), 0, 1))
    scores_hybrid_recommender.fit(weights)
    _, _, ev_map, _, _ = evaluator(scores_hybrid_recommender, data_train, data_val)
    return ev_map

scores_hybrid_study.optimize(objective, n_trials=100)

ScoresHybridRecommender: URM Detected 223 ( 1.8%) users with no interactions.
ScoresHybridRecommender: URM Detected 121 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 223 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 121 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 7049.94 column/sec. Elapsed time 3.15 sec
UserKNNCFRecommender: URM Detected 223 ( 1.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 121 ( 0.5%) items with no interactions.
Similarity column 12638 (100.0%), 7181.81 column/sec. Elapsed time 1.76 sec
RP3betaRecommender: URM Detected 223 ( 1.8%) users with no interactions.
RP3betaRecommender: URM Detected 121 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3079.04 column/sec. Elapsed time 7.22 sec
EASE_R_Recommender: URM Detected 223 ( 1.8%) users with no interactions.
EASE_R_Recommender: URM Detected 121 ( 0.5%) items with no interactions.
EASE

C:\Users\ES3\AppData\Local\Temp\ipykernel_14544\2146444983.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weights.append(trial.suggest_uniform("weight_{}".format(i), 0, 1))
[I 2024-01-10 22:04:02,482] Trial 388 finished with value: 0.07918258072254931 and parameters: {'weight_0': 0.019287375084255723, 'weight_1': 0.06014010656705178, 'weight_2': 0.7914485516733611, 'weight_3': 0.1371959272978966, 'weight_4': 0.9122349963611033, 'weight_5': 0.20437585021701493, 'weight_6': 0.563205719454579}. Best is trial 257 with value: 0.08670898442306986.
[I 2024-01-10 22:05:11,550] Trial 392 finished with value: 0.07579368966671939 and parameters: {'weight_0': 0.9890170477607235, 'weight_1': 0.0012187932746390692, 'weight_2': 0.8423123932046482, 'weight_3': 0.07251212564366584, 'weight_4': 0.8613219983391348, 'weight_5': 0.14348310648985962, 'weight_6':